In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/real-or-fake-fake-jobposting-prediction/fake_job_postings.csv


In [2]:
train=pd.read_csv("/kaggle/input/real-or-fake-fake-jobposting-prediction/fake_job_postings.csv")
train.head()


,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [3]:
#null column percentage

Col_With_NaN =train.isnull().sum() /len(train)*100
print(Col_With_NaN)

job_id                  0.000000
title                   0.000000
location                1.935123
department             64.580537
salary_range           83.959732
company_profile        18.501119
description             0.005593
requirements           15.072707
benefits               40.324385
telecommuting           0.000000
has_company_logo        0.000000
has_questions           0.000000
employment_type        19.412752
required_experience    39.429530
required_education     45.329978
industry               27.421700
function               36.101790
fraudulent              0.000000
dtype: float64


In [4]:
def MissingValue(train, Col_Dorp_thresold, Drop_raw_with_NaN = False):
    
    #for dropping column with NaN value
    Col_With_NaN =train.isnull().sum() /len(train)*100 #missing value percentage column vise
    Dropped_Columns = []
    for i in range(0,len(train.columns)):
        if Col_With_NaN[i] >= Col_Dorp_thresold:   #setting the threshold to drop columns%
            Dropped_Columns.append(train.columns[i])
    print(Dropped_Columns) #print the missing column name
    data = train.drop(Dropped_Columns,axis=1)
    
    #for dropping rows with NaN value
    if Drop_raw_with_NaN:
        Row_with_NaN =data.loc[pd.isnull(data).any(1), :].index
        print(Row_with_NaN)
        data = data.drop(Row_with_NaN,axis=0)
   
    return data

train=MissingValue(train,70,False) 
    

['salary_range']


In [5]:
train.shape



(17880, 17)

We have to convert some categorical data to numerical data

In [6]:

train['employment_type']=train['employment_type'].fillna(-1)

Remove unused categorical variable

In [7]:
train = train.drop(
        ['job_id', 'title', 'location', 'company_profile', 'description', 'requirements',
         'benefits'], axis=1).sort_index()

In [8]:
train.head()

,department,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,Marketing,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,Success,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,NaN,0,1,0,-1,NaN,NaN,NaN,NaN,0
3,Sales,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,NaN,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [9]:
numerical_features = ['telecommuting', 'has_company_logo', 'has_questions','employment_type']
label_features = [ 'required_experience', 'required_education', 'industry', 'function']

In [10]:
train['function']=train['function'].fillna(-1)
train['required_education']=train['required_education'].fillna(-1)
train['required_experience']=train['required_experience'].fillna(-1)
train['industry']=train['industry'].fillna(-1)

In [11]:
from sklearn.preprocessing import LabelEncoder
number= LabelEncoder()
train['function']=number.fit_transform(train['function'].astype('str'))
train['required_education']=number.fit_transform(train['required_education'].astype('str'))
train['required_experience']=number.fit_transform(train['required_experience'].astype('str'))
train['industry']=number.fit_transform(train['industry'].astype('str'))
train['employment_type']=number.fit_transform(train['employment_type'].astype('str'))
train['department']=number.fit_transform(train['department'].astype('str'))                                             
train.head()

,department,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,758,0,1,0,3,5,0,0,23,0
1,1161,0,1,0,2,7,0,76,8,0
2,1321,0,1,0,0,0,0,0,0,0
3,1054,0,1,0,2,6,2,23,32,0
4,1321,0,1,1,2,6,2,52,17,0


In [12]:
X=train.drop("fraudulent", axis=1)
Y = train["fraudulent"]

X.head()
Y.head()

#X.shape[0]
#Y.shape[0]

0    0
1    0
2    0
3    0
4    0
Name: fraudulent, dtype: int64

In [13]:
from sklearn.svm import SVC, LinearSVC

svc = SVC()
svc.fit(X, Y)
Y_pred = svc.predict(train)
acc_svc = round(svc.score(X, Y) * 100, 2)
acc_svc

ValueError: X.shape[1] = 10 should be equal to 9, the number of features at training time

In [14]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(X, Y)
Y_pred = logreg.predict(train)
acc_log = round(logreg.score(X, Y) * 100, 2)
acc_log

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


ValueError: X has 10 features per sample; expecting 9